# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [9]:
# your code here
orders = pd.read_csv("data/Orders/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [11]:
# your code here

# Aggregate total amount spent per customer
customer_spending = orders.groupby("CustomerID")["amount_spent"].sum().reset_index()

customer_spending.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [12]:
# Calculate the 95th and 75th percentiles
vip_threshold = customer_spending["amount_spent"].quantile(0.95)
preferred_threshold = customer_spending["amount_spent"].quantile(0.75)

print(f"VIP Threshold: {vip_threshold}")
print(f"Preferred Threshold: {preferred_threshold}")

VIP Threshold: 5840.181999999982
Preferred Threshold: 1661.64


In [13]:
# Assign customer labels

def classify_customer(spent):
    if spent > vip_threshold:
        return "VIP"
    elif spent > preferred_threshold:
        return "Preferred"
    else:
        return "Regular"


customer_spending["customer_type"] = customer_spending["amount_spent"].apply(classify_customer)


customer_spending.head()

,CustomerID,amount_spent,customer_type
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [24]:
# your code here
# Merge customer_spending with orders to get the Country information
customer_spending_with_country = customer_spending.merge(orders[['CustomerID', 'Country']].drop_duplicates(), on='CustomerID', how='left')

# Count the number of VIP customers per country
vip_counts = customer_spending_with_country[customer_spending_with_country["customer_type"] == "VIP"] \
    .groupby("Country")["CustomerID"].count().reset_index()

# Rename columns for clarity
vip_counts = vip_counts.rename(columns={"CustomerID": "VIP_count"})

# Sort in descending order to find the country with the most VIPs
vip_counts = vip_counts.sort_values(by="VIP_count", ascending=False)

# Display the country with the most VIP customers
print("Country with the most VIP customers:")
vip_counts.head(1)

Country with the most VIP customers:


,Country,VIP_count
17,United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [23]:
# your code here
# Count the number of VIP and Preferred customers per country
vip_preferred_counts = customer_spending_with_country[
    customer_spending_with_country["customer_type"].isin(["VIP", "Preferred"])
].groupby("Country")["CustomerID"].count().reset_index()

# Rename columns for clarity
vip_preferred_counts = vip_preferred_counts.rename(columns={"CustomerID": "VIP_Preferred_count"})

# Sort in descending order to find the country with the most VIP + Preferred customers
vip_preferred_counts = vip_preferred_counts.sort_values(by="VIP_Preferred_count", ascending=False)

# Display the country with the most VIP + Preferred customers
print("\nCountry with the most VIP + Preferred customers:")
vip_preferred_counts.head(1)


Country with the most VIP + Preferred customers:


,Country,VIP_Preferred_count
26,United Kingdom,932
